In [7]:
from dotenv import load_dotenv

load_dotenv()

True

In [8]:
from ipywidgets import FileUpload
from IPython.display import display

uploader = FileUpload(accept='.png.jpg', multiple=False)
display(uploader)

FileUpload(value=(), accept='.png.jpg', description='Upload')

In [9]:
print(uploader.value)

({'name': 'lelft_over.jpg', 'type': 'image/jpeg', 'size': 424988, 'content': <memory at 0x10d786b00>, 'last_modified': datetime.datetime(2026, 2, 9, 22, 56, 31, 715000, tzinfo=datetime.timezone.utc)},)


In [10]:
import base64

# Get the first (and only) uploaded file dict
uploaded_file = uploader.value[0]

# This is a memoryview
content_mv = uploaded_file["content"]

# Convert memoryview -> bytes
img_bytes = bytes(content_mv)  # or content_mv.tobytes()

# Now base64 encode
img_b64 = base64.b64encode(img_bytes).decode("utf-8")

In [16]:
from langchain.tools import tool
from typing import Dict, Any
from tavily import TavilyClient

tavily_client = TavilyClient()

@tool
def web_search(query: str) -> Dict[str, Any]:

    """Search the web for information"""

    return tavily_client.search(query)

In [24]:
system_prompt = """

You are a personal chef. The user may share a list of ingredients they have, or an image of leftover food/ingredients.

**Step 1 – Identify ingredients:** If the user provides an image, first look at it carefully and list all the foods and ingredients you can see (e.g. vegetables, proteins, dairy, herbs). Be specific about types and quantities if visible. If they only give a text list, use that as your ingredient list.

**Step 2 – Think about what to make:** Using the ingredients you identified, think about what dishes or recipes would work well with that combination. Consider cuisines, cooking methods, and common pairings.

**Step 3 – Search and suggest:** Use the web search tool to find real recipes that match those ingredients. Return recipe suggestions and give the full recipe instructions (steps, quantities, etc.) straight away for each suggestion. Do not list or show the ingredients to the user—use them only internally to find recipes.

"""

In [25]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

agent = create_agent(
    model="gpt-5-nano",
    tools=[web_search],
    system_prompt=system_prompt,
    checkpointer=InMemorySaver()
)

In [26]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

multimodal_question = HumanMessage(content=[
    {"type": "text", "text": "I have some leftovers shown in the image. What can I make with them?"},
    {"type": "image", "base64": img_b64, "mime_type": "image/png"}
])

{"messages": [multimodal_question]}

response = agent.invoke(
    {"messages": [multimodal_question]},
    config
)

print(response['messages'][-1].content)

Nice spread! From what I can see in the photo, you’ve got:

- Vegetables: eggplants, tomatoes (red and some small yellow ones), a zucchini, an onion, garlic, possibly a cauliflower head, and maybe a leafy herb. 
- Fruits: watermelon, cantaloupe, bananas.

Here are a few tasty ideas that fit well with those ingredients. I pulled real recipe options you can follow, with step-by-step instructions.

Option 1 — Ratatouille (classic French vegetable stew)
Why it fits: uses eggplant, tomatoes, zucchini, onion, garlic, and herbs; very practical for leftovers.

How to make (adapted to what you have)
1) Heat 2 tablespoons olive oil in a large skillet over medium heat.
2) Add 1 chopped onion and 1 chopped bell pepper (optional if you have one); sauté until softened, about 5 minutes.
3) Add 2–3 minced garlic cloves; cook 1 minute more until fragrant.
4) Add 1–2 cubes of eggplant; season lightly with salt; cook until browned and softened, about 6–8 minutes.
5) Add 1–2 sliced zucchini and 2–3 choppe

In [27]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content=[{'type': 'text', 'text': 'I have some leftovers shown in the image. What can I make with them?'}, {'type': 'image', 'base64': '...', 'mime_type': 'image/png'}], additional_kwargs={}, response_metadata={}, id='94678a76-a46f-403c-9ff7-b219db0fe8bb'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 4507, 'prompt_tokens': 1171, 'total_tokens': 5678, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 4352, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D7UigtADzsKQ6052Lq2nr7GAKf7Fh', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c44ad-98aa-7ab0-85cb-9116e1d9e17b-0', tool_calls=[{'name': 'web_search', 'args': {'query': 